In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yfin
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [34]:
yfin.pdr_override()

data = pd.DataFrame()

tickers = ['WIPRO.NS','^NSEI']

for t in tickers:
    
    data[t] = pdr.get_data_yahoo(t,start='2023-01-01',end='2023-01-25')['Adj Close']

print(data)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                             WIPRO.NS         ^NSEI
Date                                               
2023-01-02 00:00:00+05:30  392.084259  18197.449219
2023-01-03 00:00:00+05:30  396.074432  18232.550781
2023-01-04 00:00:00+05:30  388.742523  18042.949219
2023-01-05 00:00:00+05:30  385.949371  17992.150391
2023-01-06 00:00:00+05:30  381.061432  17859.449219
2023-01-09 00:00:00+05:30  390.837341  18101.199219
2023-01-10 00:00:00+05:30  392.084259  17914.150391
2023-01-11 00:00:00+05:30  392.882294  17895.699219
2023-01-12 00:00:00+05:30  393.530701  17858.199219
2023-01-13 00:00:00+05:30  392.932159  17956.599609
2023-01-16 00:00:00+05:30  397.620636  17894.849609
2023-01-17 00:00:00+05:30  395.376160  18053.300781
2023-01-18 00:00:00+05:30  402.209351  18165.349609
2023-01-19 00:00:00+05:30  402.408844  18107.849609
2023-01-20 00:00:00+05:30  402

In [35]:
sec_returns = np.log(data / data.shift(1))
sec_returns

,WIPRO.NS,^NSEI
Date,,
2023-01-02 00:00:00+05:30,NaN,NaN
2023-01-03 00:00:00+05:30,0.010125,0.001927
2023-01-04 00:00:00+05:30,-0.018685,-0.010454
2023-01-05 00:00:00+05:30,-0.007211,-0.002819
2023-01-06 00:00:00+05:30,-0.012746,-0.007403
2023-01-09 00:00:00+05:30,0.025331,0.013445
2023-01-10 00:00:00+05:30,0.003185,-0.010387
2023-01-11 00:00:00+05:30,0.002033,-0.001031
2023-01-12 00:00:00+05:30,0.001649,-0.002098


In [36]:
cov = sec_returns.cov()*252 #Annualize by multiplying by 252 (trading days in a year)
cov

,WIPRO.NS,^NSEI
WIPRO.NS,0.030307,0.011713
^NSEI,0.011713,0.011454


In [37]:
cov_with_market = cov.iloc[0,1]
cov_with_market

0.011713388897838543

In [38]:
market_var = sec_returns['^NSEI'].var()*252
market_var

0.011454273647280286

In [39]:
wipro_beta = cov_with_market / market_var
wipro_beta

1.0226217094629813

In [40]:
riskfree = 0.073
riskpremium = 0.07
wipro_capm_return_1 = riskfree + wipro_beta*riskpremium
wipro_capm_return_1

0.14458351966240868

In [41]:
riskfree = 0.073
riskpremium = (sec_returns['^NSEI'].mean()*252) - riskfree
wipro_capm_return = riskfree + wipro_beta*riskpremium

In [42]:
wipro_capm_return

-0.07185727937546006

In [43]:
log_returns = np.log(data / data.shift(1))
sharpe_wipro = (wipro_capm_return_1-riskfree)/(log_returns['WIPRO.NS'].std()*250**0.5)

In [44]:
sharpe_wipro

0.4128329769471662

In [45]:
lr = log_returns['WIPRO.NS'].std()*250**0.5
lr

0.17339583720214735

In [46]:
print(f"Beta: {round(wipro_beta,2)}")
print(f"Sharpe: {round(sharpe_wipro,2)}") 
print(f"CAPM Return: {round(100*wipro_capm_return)}%")

Beta: 1.02
Sharpe: 0.41
CAPM Return: -7%
